## Funciones químicas

Funciones generales que se usan para varios módulos

### Importo dependencias

In [ ]:
#Importo librerías
# import math
# import numpy as np

In [ ]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [ ]:
#Importo notebooks
from parametros import n_species,masa_species
from parametros import *

### Funciones

In [ ]:
def n_tot(n):
   #Función que calcula el número total de partículas de una mezcla
   #n: vector de nro de partículas de cada especie
   suma = 0
   for i in range(n_species):
      suma = suma + n[i]
   return suma

def masa(n):
    #Calcula la masa total de las partículas dentro de la burbuja
    m = 0.0
    for i in range(n_species):
        m = m + n[i]*masa_species[i]
    return m


def MasaMolar(Masa):
    #funcion que asigna la masa molar a cada especie
    #Masa: array que contiene n_species elementos
    #Esta función debería ser cambiada si se tienen en cuenta menos o más elementos.
    
    Masa[0]=MH2
    Masa[1]=MH
    Masa[2]=MO
    Masa[3]=MO2
    Masa[4]=MOH
    Masa[5]=masa_H2O_kgmol

    # Masa[7]=MN
    # Masa[8]=MNH
    # Masa[9]=MNH2
    # Masa[10]=MNH3
    # Masa[11]=MN2H
    # Masa[12]=MNO
    # Masa[13]=MNO2
    # Masa[14]=MN2O
    # Masa[15]=MHNO
    # Masa[16]=MN2
    # Masa[17]=MAr

    Masa[6] = MH2O2 #Antes llamado Masa[18]
    Masa[7] = MHO2 #Antes llamado Masa[19] 



def masa_media_por_particula(ntot, n):
    #Calcula la masa media por particula
    #ntot: nro total de partículas
    
    Masa = np.zeros(n_species) #Creo el vector de masas que contiene las masas de cada especi
    MasaMolar(Masa) #Inicializo las masas

    #Calculo la masa media por particula m
    M=0.0
    icont=1
    for i in range(n_species):
        M=M+(Masa[i]*n[i])/Na
    m=M/ntot

    return m
